In [ ]:
!pip install implicit

# Installations


In [ ]:
# Imports

import numpy as np
import pandas as pd
import statistics
import implicit
import pickle
import os

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from implicit.evaluation import ndcg_at_k,AUC_at_k,mean_average_precision_at_k


In [ ]:
# Importing File with Ground Truth

data = pd.read_csv('./recommendations_filtered.csv')

# Preprocess

train, test = train_test_split(data, test_size = 0.2)

train_users, train_items, train_ratings = zip(*train.values)

train_num_users = max(train_users) + 1
train_num_items = max(train_items) + 1

train_sparse = csr_matrix((train_ratings, (train_users, train_items)), shape = (train_num_users, train_num_items))

test_users, test_items, test_ratings = zip(*test.values)
test_num_users = max(test_users) + 1
test_num_items = max(test_items) + 1
test_sparse = csr_matrix((test_ratings, (test_users, test_items)), shape = (test_num_users, test_num_items))

# Fit

model = implicit.gpu.als.AlternatingLeastSquares(factors = 50)
model.fit(train_sparse)

# Test

print('NDCG ', ndcg_at_k(model = model, train_user_items = train_sparse, test_user_items = test_sparse, K = 20))
print('AUC ', AUC_at_k(model = model, train_user_items = train_sparse, test_user_items = test_sparse, K = 20))
print('MAP ', mean_average_precision_at_k(model = model, train_user_items = train_sparse, test_user_items = test_sparse, K = 20))
